<a href="https://colab.research.google.com/github/ramahasiba/NLP/blob/LangChain/Build_an_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Build an agent](https://python.langchain.com/docs/tutorials/agents/)

A system uses LLM as reasoning to determine which actions to take and the input necessary to perform the action.

## Installation

In [ ]:
!pip install -qU "langchain[groq]"
%pip install -U langgraph langchain-tavily langgraph-checkpoint-sqlite -q

## Setup

In [ ]:
!pip install -q dotenv

import os
from pprint import pprint
from dotenv import load_dotenv
import getpass

try:
  load_dotenv('.env')
except ImportError:
  print('No .env file found')

# Setup LangSmith to be able to inspect what exactly goes inside my chain or agent
os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
  os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
      prompt = "Enter the Langsmith api key:"
  )

if "LANGSMITH_PROJECT" not in os.environ:
  os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
      prompt = "Enter langsmith project name: "
  )
  if not os.environ.get("LANGSMITH_PROJECT"):
    os.environ["LANGSMITH_PROJECT"] = "default"

if "TAVILY_API_KEY" not in os.environ:
  os.environ["TAVILY_API_KEY"] = getpass.getpass(
      prompt = "Enter the Tavily API key:"
  )
os.environ["GROQ_API_KEY"] = os.getenv('GROQ_API_KEY')
os.environ["HF_TOKEN"] = os.getenv('HF_TOKEN')

Enter langsmith project name: ··········


In [ ]:
from langchain.chat_models import init_chat_model # Chat model is instance of the runnable interface
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

model_name = "llama3-70b-8192"

# Create the agent
memory = MemorySaver()
model=init_chat_model(model_name, model_provider="groq")
search = TavilySearch(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [ ]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}

input_message ={
    "role": "user",
    "content": "Hi, I am Rama I live in Nablus-Palestine."
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
  step["messages"][-1].pretty_print()

In [ ]:
input_message = {
    "role": "user",
    "content": "What's the weather where I live?"
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
  step["messages"][-1].pretty_print()

## Define tools

In [ ]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in SF?")
print(search_results)

tools = [search]

{'query': 'What is the weather in SF?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1751098875, 'localtime': '2025-06-28 01:21'}, 'current': {'last_updated_epoch': 1751098500, 'last_updated': '2025-06-28 01:15', 'temp_c': 13.3, 'temp_f': 55.9, 'is_day': 0, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png', 'code': 1030}, 'wind_mph': 7.2, 'wind_kph': 11.5, 'wind_degree': 201, 'wind_dir': 'SSW', 'pressure_mb': 1014.0, 'pressure_in': 29.93, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 90, 'cloud': 25, 'feelslike_c': 12.4, 'feelslike_f': 54.3, 'windchill_c': 10.1, 'windchill_f': 50.1, 'heatindex_c': 11.2, 'heatindex_f': 52.2, 'dewpoint_c': 10.6, 'dewpoi

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("llama3-70b-8192", model_provider="groq")

In [ ]:
query = "Hello!"
response = model.invoke([{
    "role": "user",
    "content": query
}])
response.text()

"Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?"

using .bind_tools() to enable the model to do tool calling, and to give the model those tools.

In [ ]:
model_with_tools = model.bind_tools(tools)

Calling the model with a normal message, and then take a look at both the content field as well as the tool_calls field.

In [ ]:
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool call: {response.tool_calls}")

Message content: Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?

Tool call: []


Calling the model with some input that would expect a tool to be called.

In [ ]:
query = "search for the weather in SF"

response = model_with_tools.invoke([{
    "role": "user",
    "content": query
}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'weather in SF', 'search_depth': 'basic'}, 'id': 'qe21nw4v1', 'type': 'tool_call'}]


As we can see above, the Message content is empty, and this is because we used (model called) the tool without creating the agent.


## Create the Agent

In [ ]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [ ]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
  message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?


In [ ]:
input_message = {"role": "user", "content": "Search for the weather in SF"}
response =agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
  message.pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================
Tool Calls:
  tavily_search (5spx39p9g)
 Call ID: 5spx39p9g
  Args:
    include_images: False
    query: weather in SF
    search_depth: basic
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in SF", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1751101627, 'localtime': '2025-06-28 02:07'}, 'current': {'last_updated_epoch': 1751101200, 'last_updated': '2025-06-28 02:00', 'temp_c': 13.3, 'temp_f': 55.9, 'is_day': 0,

In [ ]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
  step["messages"][-1].pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================
Tool Calls:
  tavily_search (4qbexpkgx)
 Call ID: 4qbexpkgx
  Args:
    query: weather san francisco
================================= Tool Message =================================
Name: tavily_search

{"query": "weather san francisco", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco, California", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1751101831, 'localtime': '2025-06-28 02:10'}, 'current': {'last_updated_epoch': 1751101200, 'last_updated': '2025-06-28 02:00', 'temp_c': 13.3, 'temp_f': 55.9, 'is_day': 0, 'condition': {'text': 'Mist', 'icon': '/

## Streaming Tokens

In [ ]:
for step, metadata in agent_executor.stream({"messages": [input_message]}, stream_mode="messages"):
  if metadata["langgraph_node"] == "agent" and (text := step.text()):
    print(text, end="|")


The| weather| in| San| Francisco| is| expected| to| be| mild|,| with| highs| ranging| from| the| |60|s| at| the| coast| to| the| |80|s| inland|.| There| is| a| deep| marine| layer| that| will| keep| the| coast| over|cast| all| day|,| with| afternoon| clearing| expected| along| the| Highway| |101| and| Interstate| |280| corridors|.|

## Adding in Memory
Here we pass a checkpointer to give the agent an memory to make it statefull instead of being stateless.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [ ]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
input_message={"role": "user", "content": "Hi, I am Rama"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
  step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I am Rama
================================== Ai Message ==================================

Hi Rama, it's nice to meet you! Is there something I can help you with or would you like to chat?


## Streaming Messages

In [ ]:
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

Your name is Rama.
